In [ ]:
# automatically detect and update any changed files
%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from gravipy import load_data, compute_corrections, load_geology, load_geology_map_key

## Load data & apply corrections

In [ ]:
BASE_STATION_ID = "BASE"

gdf = load_data('data', 'catalina_gravity.csv')
gdf = compute_corrections(gdf, BASE_STATION_ID)
geology = load_geology()

In [ ]:
gdf.keys()

In [ ]:
def get_geology_colors(geology_types):
    map_key_df = load_geology_map_key()
    return [map_key_df.loc[gtype]["COLOR"] for gtype in geology_types]

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))
ax.scatter(gdf['measured anomaly [mGal]'], gdf['alt [m]'], label="measured", alpha=0.7)
ax.scatter(gdf['bouguer anomaly [mGal]'], gdf['alt [m]'], label="bouguer", alpha=0.7)
ax.scatter(gdf['free air anomaly [mGal]'], gdf['alt [m]'], label="free air", alpha=0.7)
ax.set_xlabel('gravity anomaly [mGal]')
ax.set_ylabel('elevation [m]')
ax.legend()
ax.grid()
plt.show()

In [ ]:
def get_transect_data():
    transect_idxs = []
    N_TRANSECT_SITES = 73
    for i in range(N_TRANSECT_SITES):
        transect_idxs += list(gdf['site id'][gdf['site id'] == str(i)].index)
    
    UTM_ZONE_11N_EPSG = 32611
    transect_gdf = gdf.loc[transect_idxs].to_crs(UTM_ZONE_11N_EPSG)
    transect_gdf['site id'] = transect_gdf['site id'].astype(int)
    return transect_gdf.sort_values('site id')

def compute_transect_distances(gdf):
    geom = transect_gdf.geometry
    distances = [0.0] + [p0.distance(p1) for p0, p1 in zip(geom, geom[1:])]
    return np.cumsum(distances)

transect_gdf = get_transect_data()
color = get_geology_colors(transect_gdf["geology"])
x = compute_transect_distances(transect_gdf)

In [ ]:
# plot tidal correction
fig, ax = plt.subplots(nrows=3, figsize=(8, 8))
ax[0].scatter(x, transect_gdf["tidal correction [mGal]"], alpha=0.7)
ax[0].set_title("tidal correction [mGal]")
ax[1].scatter(x, transect_gdf["free air correction [mGal]"], alpha=0.7)
ax[1].set_title("free air correction [mGal]")
ax[2].scatter(x, transect_gdf["bouguer plate correction [mGal]"], alpha=0.7)
ax[2].set_title("bouguer plate correction [mGal]")
fig.set_tight_layout(True)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_xlabel('along-transect distance [m]')
ax.set_ylabel('gravity anomaly [mGal]')
# ax.scatter(x, transect_gdf["measured anomaly [mGal]"], label="measured", alpha=0.7)
# ax.scatter(x, transect_gdf['free air anomaly [mGal]'], label='free air', alpha=0.7)
ax.scatter(x, transect_gdf['bouguer anomaly [mGal]'], label='bouguer plate', alpha=0.7)
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
color = np.array((transect_gdf['bouguer anomaly [mGal]'] < 10).astype(int))
print(color)
geology.to_crs(4326).plot(ax=ax, edgecolor='k', facecolor=get_geology_colors(geology.index))
transect_gdf.to_crs(4326).plot(ax=ax, alpha=0.5, c=color, marker='.', markersize=25)
plt.show()

In [ ]:
x, y = np.array([(p.x, p.y) for p in transect_gdf.geometry]).T

print(len(x))

In [ ]:
distances = np.array(distances)
fig, ax = plt.subplots()
ax.hist(distances[(distances < 150) & (distances > 50)], bins=20, alpha=0.7)
plt.show()

In [ ]:
# Computing absolute gravity

In [ ]:
airport_fa = gdf['free air anomaly [mGal]'][gdf['site id'] == 'Catalina Airport']
campground_fa = gdf['free air anomaly [mGal]'][gdf['site id'] == 'Little Harbor Campground']
road_fa = gdf['free air anomaly [mGal]'][gdf['site id'] == 'Road Station']

airport_g = 80.70
campground_g = 46.2
road_g = 53.9

In [ ]:
print(airport_g - airport_fa)
print(campground_g - campground_fa)
print(road_g - road_fa)